# CYPLAN255
### Urban Informatics and Visualization

# Lecture 19 -- Making interactive maps
******
April 3, 2024

<img src="https://i.redd.it/wz61kqeg9ze81.jpg" width=600 align='right' title='The wreck of the Arden Craig. Francis James Mortimer, c. 1911'>

# Agenda
1. Announcements
2. Interactive Maps
3. For next time
4. Questions


# 1. Announcements

- Updated Guest Speaker Schedule:
  - NEXT WEDNESDAY, APR 13: Kuan Butts (Mapbox)

- Project groups
   - three of you: thank you!
   - rest of you: get to it

# 2. Interactive Mapping

## 2.1. Interactive Mapping?

Thoughts on when and where to use interactivity:
 - https://www.axismaps.com/guide/should-a-map-be-interactive
 - https://mapbrief.com/2017/04/06/few-interact-with-our-interactive-maps-what-can-we-do-about-it/

## 2.2. Folium

Folium is a Python wrapper around leaflet.js, one of the most popular javascript libraries for rendering interactive maps on the web.

Check out the Folium docs [here](https://python-visualization.github.io/folium/index.html).

In [ ]:
import folium

With a single line of code, Folium makes it easy to pull up an interactive map. Isn't it nice to pan and zoom?

In [ ]:
folium.Map(location=[37.870818, -122.254883])  # wurster hall

And in another single line, you can save the .html to render it in the browser, or embed it in your final project site!

In [ ]:
m = folium.Map(location=[37.870818, -122.254883])  # wurster hall
m.save("my_map.html")

Just like `contextily`, Folium let's you pick the map tile provider you want to use. Folium has the following tile provider options built in:
  - OpenStreetMap
  - Stamen Terrain
  - Stamen Toner
  - Stamen Watercolor
  - CartoDB positron
  - CartoDB dark_matter
  
Try a few out in the next cell:

In [ ]:
folium.Map(location=[37.870818, -122.254883], tiles='OpenStreetMap', attr="Map tiles by Stamen Design, under CC BY 4.0. Data by OpenStreetMap, under CC BY SA.")  # wurster hall

See [here](http://leaflet-extras.github.io/leaflet-providers/preview/) or the full list of all available leaflet tile providers. You can access like through folium like so:

In [ ]:
folium.Map(
    location=[37.870818, -122.254883],
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 
    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')  # wurster hall

Let's load our favorite storefront data again:

In [ ]:
import geopandas as gpd
storefronts = gpd.read_file('https://github.com/dillonma/storefrontindex/raw/master/all56_nACSxMSA__41860.0.geojson')

It's simple enough to add a single marker to the map:

In [ ]:
storefront = storefronts.sample(1)
coords = storefront.geometry.values[0].coords[0][::-1]
m = folium.Map(location=coords, zoom_start=10)
marker = folium.Marker(
    location=coords,
    popup="Coordinates: " + str([round(x, 6) for x in coords]) + '<br>' + "City: " + str(storefront['CITY'].values[0])
)
m.add_child(marker)
m

But plotting lots of markers is requires looping through all the points, or at least list comprehension:

In [ ]:
m = folium.Map(location=[37.870818, -122.254883], zoom_start=10)
for i, storefront in storefronts.iterrows():
    coords = storefront.geometry.coords[0][::-1]
    marker = folium.Marker(
        location=coords,
        popup="Coordinates: " + str([round(x, 6) for x in coords]) + '<br>' + "City: " + str(storefront['CITY'])
    )
    m.add_child(marker)
    if i > 100:
        break
m

With that many points, you probably don't want to render them all as markers anyways. It's too hard to click on the ones that are right on top of each other. Instead, try some of the following plugins for plotting tons of point data:

In [ ]:
from folium import plugins
m = folium.Map(location=[37.870818, -122.254883], tiles='Cartodb dark_matter', zoom_start=8)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in storefronts.geometry]

# plugins.MarkerCluster(heat_data).add_to(m)
plugins.HeatMap(heat_data).add_to(m)

m

For more on Folium, read the docs!
  - choropleths: https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps
  - styling: https://python-visualization.github.io/folium/quickstart.html#Styling-function
  - GeoPandas polgyons: https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html#Add-polygons-to-map

## 2.3. Interactive maps in plotly

Adapted Sam Maurer, who adapted this from https://plot.ly/python/mapbox-county-choropleth/

Plotly can create two different kinds of interactive map widgets: 
1. Interactive versions of GeoPandas/GeoPlot maps
2. Map data overlaid on top of Mapbox tiles

Here are the best places to start:

https://plot.ly/python/plotly-express/#maps (examples)  
https://plot.ly/python-api-reference/plotly.express.html (API referece)

The names of the Plotly Express map types are:  
`scatter_geo`  
`scatter_mapbox`  
`line_geo`  
`line_mapbox`  
`choropleth`  
`choropleth_mapbox`

As an example, we'll make a choropleth map that's overlaid on top of Mapbox tiles!

### 2.3.1. Load data

In this case, we need the geometry in GeoJSON format. GeoJSON is a web-native format, so it's easy to overlay on top of map tiles.

We'll then load the feature attributes from a separate table.

In [ ]:
import requests
res = requests.get("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json")
counties = res.json()

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

### 2.3.2. Make the map

In [ ]:
import plotly
import plotly.express as px

In [ ]:
fig = px.choropleth_mapbox(
    df, 
    geojson=counties, 
    locations='fips', 
    color='unemp',
    color_continuous_scale="Viridis",
    range_color=(0, 12),
    mapbox_style="carto-positron",
    zoom=3, 
    center = {"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
    labels={'unemp':'unemployment rate'})

fig.show()

And of course we can easily generate .html to render elsewhere:

In [ ]:
with open('choro.html', 'w+') as f:
    f.write(plotly.offline.plot(fig, include_plotlyjs='cdn', output_type='div'))

## 2.4. MapboxGL

A lot of new data viz libraries have the letters "GL" at the end of their name. This stands for "Graphics Library". This means its going to try to tap into the graphics processing unit on your computer to make it possible to render vast amounts of data/complex geometries in the browser.

You'll be using your Mapbox API token for this part of the demo.

In [ ]:
import os
from mapboxgl.utils import *
from mapboxgl.viz import *
import warnings; warnings.simplefilter('ignore')

In [ ]:
with open('data/mapbox_api_key.json', 'r') as f:
    token = json.load(f)['key']

### 2.4.1 Visualizing Point Data

In [ ]:
data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/points.csv'
df = pd.read_csv(data_url).round(3)
df.head(2)

In [ ]:
geodata = df_to_geojson(
    df, 
    properties=['Avg Medicare Payments', 'Avg Covered Charges', 'date'], 
    lat='lat', 
    lon='lon', 
    precision=3)

#### 2.4.1.1. Simple point plot

In [ ]:
viz = CircleViz(geodata, 
                access_token=token, 
                radius=2, 
                center=(-95, 40), 
                zoom=3)
viz.show()

#### 2.4.1.2. Thematic color scheme

In [ ]:
# Generate data breaks using numpy quantiles and color stops from colorBrewer
measure = 'Avg Medicare Payments'
color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(1, 9)]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz(geodata,
                access_token=token, 
                color_property="Avg Medicare Payments",
                color_stops=color_stops,
                radius=2.5,
                stroke_color='black',
                stroke_width=0.2,
                center=(-95, 40),
                zoom=3,
                below_layer='waterway-label')
viz.show()

#### 2.4.1.3. Add a scale bar and labels

In [ ]:
viz.scale = True
viz.scale_unit_system = 'imperial'
viz.label_property = "Avg Medicare Payments"
viz.stroke_width = 0
viz.label_size = 8
viz.legend_text_numeric_precision = 2
viz.show()

#### 2.4.1.4 More options for customization

In [ ]:
# Map settings
viz.style = 'mapbox://styles/mapbox/dark-v9?optimize=true'
viz.label_color = 'hsl(0, 0%, 70%)'
viz.label_halo_color = 'hsla(0, 0%, 10%, 0.75)'

# Legend settings
viz.legend_gradient = False
viz.legend_fill = '#343332'
viz.legend_header_fill = '#343332'
viz.legend_text_color = 'hsl(0, 0%, 70%)'
viz.legend_key_borders_on = False
viz.legend_title_halo_color = 'hsla(0, 0%, 10%, 0.75)'

# Scale bar settings
viz.scale = True
viz.scale_border_color = 'hsla(0, 0%, 10%, 0.75)'
viz.scale_position = 'bottom-left'
viz.scale_background_color = '#343332'
viz.scale_text_color = 'hsl(0, 0%, 70%)'

# Render map
viz.show()

#### 2.4.1.5 Create a bubble map

In [ ]:
# Generate data breaks and color stops from colorBrewer
measure_color = 'Avg Covered Charges'
color_breaks = [round(df[measure_color].quantile(q=x*0.1), 2) for x in range(2, 10)]
color_stops = create_color_stops(color_breaks, colors='Blues')

# Generate radius breaks from data domain and circle-radius range
measure_radius = 'Avg Medicare Payments'
radius_breaks = [round(df[measure_radius].quantile(q=x*0.1), 2) for x in range(2, 10)]
radius_stops = create_radius_stops(radius_breaks, 0.5, 10)

# Create the viz
viz2 = GraduatedCircleViz(geodata, 
                          access_token=token,
                          color_property="Avg Covered Charges",
                          color_stops=color_stops,
                          radius_property="Avg Medicare Payments",
                          radius_stops=radius_stops,
                          stroke_color='black',
                          stroke_width=0.5,
                          center=(-95, 40),
                          zoom=3,
                          opacity=0.75,
                          below_layer='waterway-label')
viz2.show()

#### 2.4.1.6. Create a heatmap

In [ ]:
measure = 'Avg Medicare Payments'
heatmap_color_stops = create_color_stops([0.01, 0.25, 0.5, 0.75, 1], colors='RdPu')
heatmap_radius_stops = [[0, 3], [14, 100]] # increase radius with zoom

color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(2, 10)]
color_stops = create_color_stops(color_breaks, colors='Spectral')

heatmap_weight_stops = create_weight_stops(color_breaks)

# Create the heatmap 
viz3 = HeatmapViz(geodata, 
                  access_token=token,
                  weight_property="Avg Medicare Payments",
                  weight_stops=heatmap_weight_stops,
                  color_stops=heatmap_color_stops,
                  radius_stops=heatmap_radius_stops,
                  opacity=0.8,
                  center=(-95, 40),
                  zoom=3,
                  below_layer='waterway-label')
viz3.show()

#### 2.4.1.7. Marker clusters

In [ ]:
# Create a clustered circle map
color_stops = create_color_stops([1, 10, 25, 50, 75, 100], colors='YlOrBr')

viz4 = ClusteredCircleViz(geodata, 
                          access_token=token,
                          color_stops=color_stops,
                          stroke_color='black',
                          radius_stops=[[1, 5], [10, 10], [50, 15], [100, 20]],
                          radius_default=2,
                          cluster_maxzoom=10,
                          cluster_radius=30,
                          label_size=12,
                          opacity=0.9,
                          center=(-95, 40),
                          zoom=3)
viz4.show()

#### 2.4.1.8. Save to HTML file for distribution

In [ ]:
with open('viz4.html', 'w') as f:
    f.write(viz4.create_html())

### 2.4.2. Choropleths, again

#### 2.4.2.1. Basic Choropleth

In [ ]:
# create choropleth from polygon features stored as GeoJSON
viz = ChoroplethViz('https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/us-states.geojson', 
                    access_token=token,
                    color_property='density',
                    color_stops=create_color_stops([0, 50, 100, 500, 1500], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='rgb(128,0,38)',
                    line_width=1,
                    line_opacity=0.9,
                    opacity=0.8,
                    center=(-96, 37.8),
                    zoom=3,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz.show()

#### 2.4.2.2. Choropleths with match-type color scheme from GeoJSON source

In [ ]:
match_color_stops = [['Massachusetts', 'rgb(46,204,113)'],
                     ['Utah', 'rgb(231,76,60)'],
                     ['California', 'rgb(142,68,173)']]

viz = ChoroplethViz('https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/us-states.geojson', 
                    access_token=token,
                    color_property='name', 
                    color_stops=match_color_stops, 
                    color_function_type='match', 
                    color_default='rgba(52,73,94,0.5)', 
                    opacity=0.8, 
                    center=(-96, 37.8), 
                    zoom=3, 
                    below_layer='waterway-label')
viz.show()

#### 2.4.2.3. Add 3-D Extrusion

In [ ]:
# adjust view angle
viz.bearing = -15
viz.pitch = 45

# add extrusion to viz using interpolation keyed on density in GeoJSON features
viz.height_property = 'density'
viz.height_stops = create_numeric_stops([0, 50, 100, 500, 1500, 5000], 0, 500000)
viz.height_function_type = 'interpolate'

# render again
viz.show()

The data and geometries do not need to come from the same place! Here we supply our own data from `pandas`, and point the `ChoroplethViz()` method to a mapbox URL which has geometries for US States

In [ ]:
data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/2010_us_population_by_postcode.csv'
df = pd.read_csv(data_url).round(3)
df.head(2)

In [ ]:
measure = '2010 Census Population'
dimension = 'Zip Code ZCTA'
data = df[[dimension, measure]].groupby(dimension, as_index=False).mean()
color_breaks = [round(data[measure].quantile(q=x*0.1), 2) for x in range(2,11)]
color_stops = create_color_stops(color_breaks, colors='PuRd')
data = json.loads(data.to_json(orient='records'))

In [ ]:
viz = ChoroplethViz(data, 
                    access_token=token,
                    vector_url='mapbox://rsbaumann.bv2k1pl2',
                    vector_layer_name='2016_us_census_postcode',
                    vector_join_property='postcode',
                    data_join_property=dimension,
                    color_property=measure,
                    color_stops=color_stops,
                    line_color = 'rgba(0,0,0,0.05)',
                    line_width = 0.5,
                    opacity=0.7,
                    center=(-95, 45),
                    zoom=2,
                    below_layer='waterway-label',
                    legend_key_shape='contiguous-bar')
viz.show()

# 3. Questions?

# 4. For next time